In [11]:
# テキストコーパスをチャンクに分割
with open('code_gate.py', 'r', encoding='utf-8') as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(text)

In [ ]:
print(type(texts))
print("--------")
print(len(texts))
print("--------")
print(texts[0])
print("--------")
print(texts[1])

In [13]:
# パッセージのベクトル化
from langchain_huggingface import HuggingFaceEmbeddings

# HuggingFaceのモデルを使用
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large', model_kwargs={'device': 'cpu'})

# OpenAIのモデルを使用
#embeddings = OpenAIEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS

# データベースの保存
db = FAISS.from_texts(texts, embeddings)
db.save_local('code.db')

In [15]:
from langchain_community.vectorstores import FAISS

# 保存したデータベースの読み込み
db = FAISS.load_local('code.db',embeddings, allow_dangerous_deserialization=True)

In [ ]:
similarity_sample = db.similarity_search("AND")
print(len(similarity_sample))
print("--------")
print(type(similarity_sample[0]))
print("--------")
print(similarity_sample[0].page_content)

In [17]:
# 検索器の構築
retriever = db.as_retriever()   # 検索文書数 4（デフォルト）

In [18]:
# モデルの準備
import os
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = 'sk-proj-*****'

openai_llm = ChatOpenAI(model_name="gpt-4.1-nano")
qa = RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    return_source_documents=True,
    )

In [ ]:
# 実行例
q = "どんなプログラミング言語で書かれていますか？"
ans = qa.invoke(q)
print(q)
print(ans['result'])

print("--------------------------")

q = "定義されている関数を全てリストアップして下さい"
ans = qa.invoke(q)
print(q)
print(ans['result'])

print("--------------------------")

q = "AND関数は何回呼ばれますか？"
ans = qa.invoke(q)
print(q)
print(ans['result'])

print("--------------------------")

q = "このプログラムがやっていることは何ですか？"
ans = qa.invoke(q)
print(q)
print(ans['result'])